# Miembros del grupo "Los hiperparámetros" 🙍‍♂️🙍‍♀️🙍‍♂️

- MIGUEL GONZÁLEZ GARCÍA
- ROSA LÓPEZ ESCALONA
- JAVIER QUESADA PAJARES

---

# MODELO HÍBRIDO: BNMF + SVD (Public Score: 1.232)
---

# Imports 🔧

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from surprise import SVD, Dataset, Reader

# Preprocesamiento del dataset 🧹

In [5]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

global_mean = train_data['rating'].mean()
print(f"Media global de ratings: {global_mean:.4f}")

Media global de ratings: 7.6047


In [6]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [7]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [5]:
#train_data, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

## Creación de matriz dispersa 🧩

In [ ]:
unique_users = train_data['user'].unique()
unique_items = train_data['item'].unique()

user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

num_users = len(unique_users)
num_items = len(unique_items)

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")

Número de usuarios: 73456
Número de ítems: 171171


In [ ]:
# Crea la matriz dispersa
rows = train_data['user'].map(user_to_index).values
cols = train_data['item'].map(item_to_index).values
ratings = train_data['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (73456, 171171)
Número de ratings no nulos en la matriz: 390351


# Entrenamiento BNMF (con bias) 🏋️

In [ ]:
# Hiperparámetros
k = 20  # Número de factores latentes
lambda_ = 0.02  # Regularización
learning_rate = 0.002  # Tasa de aprendizaje
num_epochs = 50  # Número de iteraciones

mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Factores latentes inicializados: U ((73456, 20)), V ((171171, 20))
Sesgos inicializados: bu ((73456,)), bi ((171171,))


In [ ]:
def train_bnmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]

            # Predicción incluyendo el sesgo
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])

            # Actualización de los factores latentes
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Forzar no-negatividad
            U[i, :] = np.maximum(U[i, :], 0)
            V[j, :] = np.maximum(V[j, :], 0)

            # Costo regularizado
            total_cost += error**2 + (lambda_ / 2) * (
                np.linalg.norm(U[i, :])**2 +
                np.linalg.norm(V[j, :])**2 +
                bu[i]**2 + bi[j]**2
            )

        print(f"Época {epoch + 1}/{num_epochs} - Costo total: {total_cost:.4f}")

    return U, V, bu, bi


In [ ]:
print("Iniciando entrenamiento BNMF...")
U, V, bu, bi = train_bnmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")


Iniciando entrenamiento BNMF...
Época 1/50 - Costo total: 1260646.8514
Época 2/50 - Costo total: 1194846.7935
Época 3/50 - Costo total: 1157408.4361
Época 4/50 - Costo total: 1130087.5177
Época 5/50 - Costo total: 1108180.0322
Época 6/50 - Costo total: 1089699.7672
Época 7/50 - Costo total: 1073602.1324
Época 8/50 - Costo total: 1059263.8631
Época 9/50 - Costo total: 1046281.7793
Época 10/50 - Costo total: 1034380.1185
Época 11/50 - Costo total: 1023361.3034
Época 12/50 - Costo total: 1013078.6270
Época 13/50 - Costo total: 1003419.7170
Época 14/50 - Costo total: 994296.1939
Época 15/50 - Costo total: 985636.8998
Época 16/50 - Costo total: 977383.1568
Época 17/50 - Costo total: 969485.7243
Época 18/50 - Costo total: 961902.1431
Época 19/50 - Costo total: 954595.1160
Época 20/50 - Costo total: 947532.2440
Época 21/50 - Costo total: 940684.4110
Época 22/50 - Costo total: 934025.8708
Época 23/50 - Costo total: 927533.3188
Época 24/50 - Costo total: 921185.8150
Época 25/50 - Costo total: 9

# Entrenamiento SVD 🏋️

In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(train_data[['user', 'item', 'rating']], reader)

trainset = data.build_full_trainset()

# División en conjunto de entrenamiento y validación
#trainset, valset = train_test_split(data, test_size=0.2, random_state=42)

# Configuración del modelo SVD
svd = SVD(n_factors=20, n_epochs=50, biased=True, lr_all=0.002, reg_all= 0.02, random_state=42)

print("Entrenando el modelo SVD...")
svd.fit(trainset)
print("Modelo SVD entrenado correctamente.")

Entrenando el modelo SVD...
Modelo SVD entrenado correctamente.


# Predicciones y generación de CSV 🧠

In [ ]:
# combinacion media de las pred de BNMF y SVD
def predict_rating(uid, iid, U, V, bu, bi, mu):
    try:
        # Predicción usando SVD
        pred_svd = svd.predict(uid, iid).est
    except:
        pred_svd = None

    # Predicción usando PMF con bias
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]
        pred_pmf = mu + bu[user_idx] + bi[item_idx] + np.dot(U[user_idx, :], V[item_idx, :])
    else:
        pred_pmf = None

    # Promedio de predicciones si ambas son válidas
    if pred_svd is not None and pred_pmf is not None:
        pred = (pred_svd + pred_pmf) / 2
    elif pred_svd is not None:
        pred = pred_svd
    elif pred_pmf is not None:
        pred = pred_pmf
    else:
        pred = mu  # Si ambos fallan, usar la media global

    return f"{round(max(1, min(10, pred)))}.0"

In [ ]:
print("Generando predicciones finales...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid, iid, U, V, bu, bi, mu)
    predictions.append((row_id, pred_rating))

predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

output_filename = "predictions_bnmf_svd.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado correctamente.")

Generando predicciones finales...
Archivo 'predictions_bnmf_svd.csv' generado correctamente.
